# Keras tuner- decide Number of Hidden Layers and Neuron in Neural Network

In [57]:
import pandas as pd
import keras
from keras import layers
from kerastuner.tuners import RandomSearch

In [58]:
df=pd.read_csv('Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [59]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [60]:
y.fillna(y.mean(),inplace=True)
y.isnull().sum()

0

In [61]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

### Hyperparameter
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [62]:
def build_model(hp):
    model=keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):   # 2, 20 is range 2-20
        model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                           min_value=32,
                                           max_value=512,
                                           step=32),
                              activation='relu'))
        model.add(layers.Dense(1, activation='linear'))
        model.compile(
                    optimizer=keras.optimizers.Adam(
                    hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                loss='mean_absolute_error',
                metrics=['mean_absolute_error'])
    return model

In [63]:
tuner=RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='projec2',
    project_name='Air Quality Index')

In [64]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [65]:
# Train test split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [66]:
tuner.search(X_train,y_train,epochs=50,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 30s]
val_mean_absolute_error: 46.28301239013672

Best val_mean_absolute_error So Far: 42.66024653116862
Total elapsed time: 00h 01m 53s
INFO:tensorflow:Oracle triggered exit


In [67]:
tuner.results_summary()

Results summary
Results in projec2\Air Quality Index
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 9
units_0: 480
learning_rate: 0.01
units_1: 288
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
Score: 42.66024653116862
Trial summary
Hyperparameters:
num_layers: 2
units_0: 64
learning_rate: 0.01
units_1: 288
units_2: 480
units_3: 64
units_4: 192
units_5: 224
units_6: 160
units_7: 352
units_8: 384
Score: 43.142584482828774
Trial summary
Hyperparameters:
num_layers: 13
units_0: 480
learning_rate: 0.001
units_1: 160
units_2: 224
units_3: 320
units_4: 352
units_5: 352
units_6: 128
units_7: 352
units_8: 384
units_9: 256
units_10: 32
units_11: 32
units_12: 32
Score: 45.41237004597982
Trial summary
Hyperparameters:
num_layers: 19
units_0: 256
learning_rate: 0.001
units_1: 320
units_2: 64
units_3: 480
units_4: 384
units_5: 64
units_6: 96
units_7: 352
units_8: 416
units_9: 416
units_10: 64
units_11: 288
units_12: 64
units_13: 32
units_14: 